In [4]:
!pip install jieba
!pip install pyldavis

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [48]:
import os
import sys

import pandas as pd

filename='testv3.csv'
datapath = os.getcwd() + '\\'+ filename
outputpath = os.getcwd() + '\\'+ 'output_tfidf.csv'
setdictpath = os.getcwd() + '\\'+ 'dict.txt.big'
userdictpath  = os.getcwd() + '\\'+ 'Custom_dict_utf8.txt'
stopword_path= os.getcwd() + '\\'+ 'stopword_tw3.txt'


df = pd.read_csv(datapath,encoding='UTF-8')

In [49]:
df.head()

,article_id,article_content
0,M.1520174009.A.D36,即日起 ~ 03/28活動期間 至全台歐舒丹櫃點憑兌換簡訊即贈清泉水合體驗體 清泉水合凍 ...
1,M.1520164202.A.A76,是 1.膚質 [夏天是中性肌/容易曬黑(目前有使用酸類)] 2.訴求 日常使用 這裡定義一下...
2,M.1520133561.A.ACF,看來霓淨思面膜有蟲不是個案了 霓淨思N3杏仁酸淨膚亮白面膜 製造日期2017.09.29 也...
3,M.1520132013.A.F8E,如題 眉尾的部分覺得不夠密 看上去有點短 請教一下有什麼方法 希望可以大概簡述一下 ...
4,M.1520091559.A.FB7,一直以來DHC我都只有用很基礎的 卸妝油 洗臉 護髮油 最近想捨棄專櫃品牌改看看DHC的保養...


In [50]:
df.shape

(20158, 2)

In [51]:
import jieba

from jieba.analyse import *

setdictpath = os.getcwd() + '\\'+ 'dict.txt.big'
userdictpath  = os.getcwd() + '\\'+ 'user_dict2.txt'
stopword_path= os.getcwd() + '\\'+ 'stopword_tw3.txt'

#dictionary path assign
jieba.set_dictionary(setdictpath)
jieba.load_userdict(userdictpath) 
jieba.analyse.set_stop_words(stopword_path)

'''
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords 

def seg_sentence(sentence):  
    sentence_seged = jieba.cut(sentence.strip())  
    stopwords = stopwordslist(stopword_path)  # 这里加载停用词的路径  
    outstr = ''  
    for word in sentence_seged:  
        if word not in stopwords:  
            if word != '\t':  
                outstr += word  
                outstr += " "  
    return outstr  
'''

def jiebaclearText(text):
    mywordlist = []
    seg_list = jieba.cut(text, cut_all=False)
    liststr="/ ".join(seg_list)
    f_stop = open(stopword_path, encoding='utf-8')
    try:
        f_stop_text = f_stop.read( )
        #f_stop_text=unicode(f_stop_text,'utf-8')
    finally:
        f_stop.close( )
    f_stop_seg_list=f_stop_text.split('\n')
    for myword in liststr.split('/'):
        if not(myword.strip() in f_stop_seg_list) and len(myword.strip())>1:
            mywordlist.append(myword)
    return ''.join(mywordlist)




def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))

Building prefix dict from D:\#III Data\ptt_BeautySalon_data\wordcut\dict.txt.big ...
Loading model from cache Z:\TEMP\jieba.u3c87b4e3d42b8cafe840582fd0af9482.cache
Loading model cost 0.656 seconds.
Prefix dict has been built succesfully.


In [52]:
df["content_cutted"] = df.article_content.apply(jiebaclearText)

In [53]:
df.content_cutted.head(10)

0    即日 03 28 期間 歐舒丹 櫃點 簡訊 即贈 清泉 水合 清泉 水合 1.5 ML 清泉...
1     膚質 夏天 性肌 曬黑 酸類 訴求 日常 定義 日常 上午 00 下午 00 之間 騎車 ...
2     霓淨思 有蟲 個案 霓淨思 N3 杏仁 淨膚 亮白 製造 日期 2017.09 29 這片...
3                                         眉尾 請教 有什麼 簡述
4     DHC 基礎 卸妝油 洗臉 護髮 捨棄 DHC 保養品 前天 先買 化妝水 活膚 想要 請...
5     植村秀 潔顏油 出新 XD 剛在 Facebook 截圖 潔顏油 轉載 FB 介紹 截圖 ...
6     晚上 想要 詢問 身為 粉刺 工作 前年 12 莫名 爆痘 就醫 胡亂 保養品 三月 就醫...
7    懶人 終於 感覺到 歲月 隨便 擦擦 絲瓜水 平安 一生 年紀 這兩天 幾家 試用品 有時候...
8     好久 sabon oh baby 評價 第一次 sabon 身體 滑滑 浪費 瞬效 護髮 ...
9     過度 攝取 精緻 健康 隱憂 這件 這幾年 越來越 過度 攝取 包括 網誌 圖文 好讀 關...
Name: content_cutted, dtype: object

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

n_features = 1000

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(df.content_cutted)


In [57]:
from sklearn.decomposition import LatentDirichletAllocation

n_components = 5
lda = LatentDirichletAllocation(n_components=n_components, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [58]:
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=5, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [59]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [60]:
n_top_words = 20

In [61]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
卸妝 xd 護唇膏 划算 卸妝油 台灣 修護 保濕 優惠 拿到 30ml 卸妝水 50ml 想要 第一次 保養品 蘭蔻 本來 試用品 護手霜
Topic #1:
防曬 身體 清爽 10 噴霧 雅漾 屈臣氏 隔離 spf50 曬黑 orbis xd 太陽 50 潤色 防水 12 理膚寶水 卸妝 蜜粉
Topic #2:
粉刺 保濕 化妝水 毛孔 美白 玻尿酸 杏仁 dr sent 精華液 qq 保養品 膚質 xd 鼻子 角鯊 android 寶拉 wu 出油
Topic #3:
保濕 化妝水 吸收 xd 保養品 精華液 清爽 回購 夏天 乳霜 冬天 滋潤 濕敷 敏感 美白 洗面乳 按摩 過敏 拿來 脫皮
Topic #4:
醫生 保養品 過敏 皮膚科 粉刺 角質 身體 按摩 美容師 改善 xd 雷射 酒糟 膠原 這是 臉部 洗臉 經驗 好了 不用



In [62]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      34.726673        1       1  0.084905  0.031215
4      19.575031        1       2  0.061775 -0.016281
0      18.604661        1       3 -0.079264 -0.206631
2      17.526592        1       4  0.170088  0.067484
1       9.567043        1       5 -0.237503  0.124214, topic_info=     Category          Freq   Term         Total  loglift  logprob
term                                                              
945   Default  10082.000000     防曬  10082.000000  30.0000  30.0000
712   Default  11042.000000     粉刺  11042.000000  29.0000  29.0000
239   Default   4957.000000     卸妝   4957.000000  28.0000  28.0000
926   Default   3794.000000     醫生   3794.000000  27.0000  27.0000
164   Default  16097.000000     保濕  16097.000000  26.0000  26.0000
235   Default   8726.000000    化妝水   8726.000000  25.0000  25.0000
600   Default   4280.000000     清爽   4280.000000  24.0000  24.0000
269   Default   5159.000000     吸收   5159.000000  23.0000  23.0000
546   Default   3674.000000     毛孔   3674.000000  22.0000  22.0000
878   Default   4008.000000     身體   4008.000000  21.0000  21.0000
845   Default   2142.000000    護唇膏   2142.000000  20.0000  20.0000
525   Default   1832.000000     杏仁   1832.000000  19.0000  19.0000
50    Default   1805.000000     dr   1805.000000  18.0000  18.0000
291   Default   3138.000000     回購   3138.000000  17.0000  17.0000
645   Default   2466.000000    玻尿酸   2466.000000  16.0000  16.0000
676   Default   1741.000000    皮膚科   1741.000000  15.0000  15.0000
760   Default   4952.000000     美白   4952.000000  14.0000  14.0000
285   Default   1394.000000     噴霧   1394.000000  13.0000  13.0000
74    Default   1873.000000   sent   1873.000000  12.0000  12.0000
720   Default   5306.000000    精華液   5306.000000  11.0000  11.0000
954   Default   1653.000000     雅漾   1653.000000  10.0000  10.0000
5     Default   2981.000000     10   2981.000000   9.0000   9.0000
145   Default   3416.000000     乳霜   3416.000000   8.0000   8.0000
605   Default   2725.000000     滋潤   2725.000000   7.0000   7.0000
241   Default   1254.000000    卸妝油   1254.000000   6.0000   6.0000
625   Default   2807.000000     濕敷   2807.000000   5.0000   5.0000
354   Default   1206.000000    屈臣氏   1206.000000   4.0000   4.0000
998   Default   1531.000000     鼻子   1531.000000   3.0000   3.0000
306   Default   3675.000000     夏天   3675.000000   2.0000   2.0000
759   Default   1180.000000    美容師   1180.000000   1.0000   1.0000
...       ...           ...    ...           ...      ...      ...
81     Topic5    304.092805    spf    304.902034   2.3442  -5.5371
638    Topic5    332.053177    物理性    334.013503   2.3410  -5.4491
64     Topic5    683.353409  orbis    710.258421   2.3082  -4.7274
947    Topic5    571.071978     防水    594.494003   2.3067  -4.9069
16     Topic5    453.761895     16    478.244023   2.2943  -5.1368
28     Topic5    395.412493     22    428.382856   2.2668  -5.2745
41     Topic5    414.283760     85    455.036687   2.2530  -5.2278
19     Topic5    403.374699     19    449.885761   2.2377  -5.2545
623    Topic5    586.526049     潤色    656.471729   2.2342  -4.8802
503    Topic5    301.714300     曬傷    341.868055   2.2219  -5.5449
320    Topic5    589.685753     太陽    677.461253   2.2081  -4.8748
507    Topic5    457.479099   曼秀雷敦    533.658833   2.1928  -5.1287
32     Topic5    375.404536     26    447.948562   2.1702  -5.3264
17     Topic5    256.130061     17    308.614687   2.1604  -5.7087
805    Topic5    514.600131     蜜粉    630.220948   2.1442  -5.0110
285    Topic5    980.622084     噴霧   1394.114137   1.9950  -4.3662
952    Topic5    772.691057     隔離   1076.489477   2.0153  -4.6045
354    Topic5    820.481217    屈臣氏   1206.672327   1.9611  -4.5445
954    Topic5    913.247455     雅漾   1653.242954   1.7534  -4.4374
37     Topic5    587.921138     50    877.303255   1.9466  -4.8778
5      Topic5   1085.255681 

In [44]:
data = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.show(data)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [23/Mar/2018 15:05:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 15:05:17] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 15:05:17] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 15:05:17] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 15:05:17] code 404, message Not Found
127.0.0.1 - - [23/Mar/2018 15:05:17] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Mar/2018 15:05:17] code 404, message Not Found
127.0.0.1 - - [23/Mar/2018 15:05:17] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Mar/2018 15:06:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 15:06:37] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 15:06:37] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 15:06:37] "GET /LDAvis.js HTTP/1.1" 200 -



stopping Server...


In [63]:
n_components = 10 
lda = LatentDirichletAllocation(n_components=n_components, max_iter=50,
                                              learning_method='online',
                                              learning_offset=50.,
                                              random_state=0)
lda.fit(tf)
print_top_words(lda, tf_feature_names, n_top_words)
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

Topic #0:
卸妝 按摩 卸妝油 卸妝水 洗臉 護膚 睫毛 導入 洗面乳 眼唇 導入儀 凝露 卸完 植村秀 底妝 平常 彩妝 粉餅 xd 卸妝液
Topic #1:
防曬 清爽 spf50 隔離 曬黑 噴霧 太陽 身體 潤色 pa 夏天 防水 orbis 蜜粉 係數 補擦 杜鵑花 理膚寶水 臉部 xd
Topic #2:
蘭蔻 精露 倩碧 修護 ipsa 特潤 雅詩蘭黛 奇蹟 保養品 乳霜 緊緻 母親節 skii 50ml 這組 周年慶 活膚霜 淡斑 kiehls 雪肌精
Topic #3:
粉刺 洗面乳 毛孔 角質 洗臉 洗臉機 鼻子 刷頭 溫和 xd 鼻頭 杏仁 控油 敏感 膚質 出油 收斂 抗痘 黑頭 洗顏
Topic #4:
雷射 膠原 保養品 美白 台灣 角質 實驗 化妝品 青春痘 導致 這類 天然 改善 一種 含有 添加 防腐劑 預防 刺激 臉部
Topic #5:
保濕 化妝水 吸收 保養品 xd 精華液 濕敷 夏天 清爽 冬天 乳霜 脫皮 膚質 敏感 過敏 美白 早上 滋潤 晚上 混合
Topic #6:
身體 xd 護唇膏 回購 台灣 滋潤 精油 護手霜 嘴唇 dhc 寶拉 拿來 沐浴乳 冬天 頭髮 滋潤度 保濕 夏天 天然 吸收
Topic #7:
醫生 過敏 粉刺 皮膚科 保養品 qq 痘疤 美容師 改善 酒糟 脫皮 看醫生 臉頰 情況 好了 藥膏 困擾 敏感 紅腫 晚上
Topic #8:
保濕 玻尿酸 精華液 化妝水 美白 dr 雅漾 角鯊 康是美 wu 杏仁 薇姿 醫美節 醫美 凝露 修護 霓淨思 haba 肌研 乳霜
Topic #9:
xd 10 sent 發文 剛剛 android qq 第一次 12 po 昨天 手機 推銷 優惠 20 詢問 台灣 保養品 收到 結帳



PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      22.644654        1       1  0.031952 -0.110955
9      12.632067        1       2 -0.038501  0.198641
7      11.534056        1       3 -0.204346  0.033333
6      11.518688        1       4 -0.025010 -0.067809
3       8.717067        1       5 -0.077207 -0.130975
2       8.320621        1       6  0.217447  0.170228
8       7.140734        1       7  0.286122  0.010813
4       6.799871        1       8 -0.189734  0.180273
1       5.563790        1       9  0.031301 -0.138799
0       5.128453        1      10 -0.032025 -0.144749, topic_info=     Category          Freq Term         Total  loglift  logprob
term                                                            
945   Default  10097.000000   防曬  10097.000000  30.0000  30.0000
712   Default  11071.000000   粉刺  11071.000000  29.0000  29.0000
164   Default  16128.000000   保濕  16128.000000  28.0000  28.0000
239   Default   5086.000000   卸妝   5086.000000  27.0000  27.0000
444   Default   3739.000000   按摩   3739.000000  26.0000  26.0000
235   Default   8753.000000  化妝水   8753.000000  25.0000  25.0000
720   Default   5319.000000  精華液   5319.000000  24.0000  24.0000
926   Default   3732.000000   醫生   3732.000000  23.0000  23.0000
546   Default   3687.000000   毛孔   3687.000000  22.0000  22.0000
580   Default   3453.000000  洗面乳   3453.000000  21.0000  21.0000
645   Default   2488.000000  玻尿酸   2488.000000  20.0000  20.0000
760   Default   5004.000000   美白   5004.000000  19.0000  19.0000
878   Default   4015.000000   身體   4015.000000  18.0000  18.0000
577   Default   3425.000000   洗臉   3425.000000  17.0000  17.0000
820   Default   3083.000000   角質   3083.000000  16.0000  16.0000
50    Default   1828.000000   dr   1828.000000  15.0000  15.0000
600   Default   4279.000000   清爽   4279.000000  14.0000  14.0000
845   Default   2175.000000  護唇膏   2175.000000  13.0000  13.0000
269   Default   5156.000000   吸收   5156.000000  12.0000  12.0000
625   Default   2802.000000   濕敷   2802.000000  11.0000  11.0000
241   Default   1289.000000  卸妝油   1289.000000  10.0000  10.0000
802   Default   1469.000000   蘭蔻   1469.000000   9.0000   9.0000
291   Default   3146.000000   回購   3146.000000   8.0000   8.0000
676   Default   1713.000000  皮膚科   1713.000000   7.0000   7.0000
263   Default   2813.000000   台灣   2813.000000   6.0000   6.0000
905   Default   4660.000000   過敏   4660.000000   5.0000   5.0000
954   Default   1654.000000   雅漾   1654.000000   4.0000   4.0000
605   Default   2728.000000   滋潤   2728.000000   3.0000   3.0000
960   Default   1185.000000   雷射   1185.000000   2.0000   2.0000
145   Default   3412.000000   乳霜   3412.000000   1.0000   1.0000
...       ...           ...  ...           ...      ...      ...
528   Topic10    434.916083  植村秀    435.826107   2.9683  -4.5557
242   Topic10    389.917537  卸妝液    390.827510   2.9680  -4.6649
688   Topic10    345.871255  睫毛膏    346.781226   2.9677  -4.7848
853   Topic10    329.602677   貝德    430.939357   2.7023  -4.8330
547   Topic10    211.480846   氣墊    298.575469   2.6255  -5.2767
444   Topic10   2588.635766   按摩   3739.307457   2.6026  -2.7720
80    Topic10    365.922548  spa    540.872896   2.5796  -4.7285
804   Topic10    302.199166   蜜妮    474.830705   2.5185  -4.9198
381   Topic10    415.314923   彩妝    706.939711   2.4385  -4.6018
714   Topic10    410.531763   粉餅    714.697759   2.4160  -4.6134
467   Topic10    190.524175   擦拭    340.510585   2.3897  -5.3811
44    Topic10    243.013183   bb    455.302093   2.3425  -5.1378
793   Topic10    218.845049   萊雅    454.670494   2.2392  -5.2425
541   Topic10    156.332054   殘留    342.989329   2.1846  -5.5789
476   Topic10    146.850150   放鬆    341.500119   2.1264  -5.6415
377   Topic10    434.859391   底妝   1116.431197   2.0275  -4.5559
434   Topic10    162.732647   拉提    431.236183   1.9958  -5.5388
443   Topic10    132.147269   按壓    373.621812   1.9310  -5

In [47]:
data = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
pyLDAvis.show(data)


Note: if you're in the IPython notebook, pyLDAvis.show() is not the best command
      to use. Consider using pyLDAvis.display(), or pyLDAvis.enable_notebook().
      See more information at http://pyLDAvis.github.io/quickstart.html .

You must interrupt the kernel to end this command

Serving to http://127.0.0.1:8889/    [Ctrl-C to exit]


127.0.0.1 - - [23/Mar/2018 16:08:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 16:08:56] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 16:08:56] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Mar/2018 16:08:56] "GET /LDAvis.js HTTP/1.1" 200 -



stopping Server...
